# How to perform bulk insert, update, and delete operations

Performing bulk insert, update, and delete operations with C# and Dapper can help you improve the performance and efficiency of your data access layer. Dapper is a micro-ORM that simplifies the data access code and reduces the boilerplate code. You can use Dapper's extension methods, query builders, and custom mappers to perform bulk operations from a collection of objects to a SQL Server table.

To illustrate this, let's assume that we have a collection of <code>Product</code> objects that we want to perform bulk operations on. The <code>Product</code> class and the Production.Product table have the same schema as in the previous example. We can use the following code snippets to perform bulk operations using Dapper:



In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:System.ComponentModel"
#r "nuget:Dapper"
#r "nuget:Z.Dapper.Plus"

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.4 System.ComponentModel, 4.3.0 Z.Dapper.Plus, 7.2.1

In [3]:
using Microsoft.Data.SqlClient;
using System;
using System.Collections.Generic;
using Dapper;
using Z.Dapper.Plus;

public class Product
{    
    public int ProductID { get; set; }
    public string Name { get; set; }
    public string ProductNumber { get; set; }
    public string Color { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public int SafetyStockLevel { get; set; }
    public int ReorderPoint { get; set; }
    public int DaysToManufacture { get; set; }
    public DateTime SellStartDate { get; set; }
}

__Bulk insert:__

In [4]:
var rand = new Random();
var productsToInsert = new List<Product>()
{
    new Product() { Name = "Bike Helmet-" + rand.Next(1000, 9999), ProductNumber = "BK-H01-" + rand.Next(1000, 9999),
                    Color = "Red", StandardCost = 15.50m, ListPrice = 34.99m, SafetyStockLevel = 999, 
                    ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) },
    new Product() { Name = "Water Bottle-" + rand.Next(1000, 9999), ProductNumber = "WB-H01-" + rand.Next(1000, 9999),
                    Color = "Blue", StandardCost = 1.00m, ListPrice = 3.49m, SafetyStockLevel = 999, 
                    ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) },
    new Product() { Name = "Gloves-" + rand.Next(1000, 9999), ProductNumber = "GL-H01-" + rand.Next(1000, 9999),
                    Color = "Black", StandardCost = 5.00m, ListPrice = 9.99m, SafetyStockLevel = 999, 
                    ReorderPoint = 777, DaysToManufacture = 1, SellStartDate = new DateTime(2000, 1, 2) }
};

List<Product> newPoducts;
using (var connection = new SqlConnection(connectionString))
{
    // Open the connection
    connection.Open();

    // Configure Dapper Plus to map the Product entity to the Product table and specify the identity column
    DapperPlusManager.Entity<Product>().Table("Production.Product").Identity(x => x.ProductID);

    // Perform bulk insert operation
    connection.BulkInsert(productsToInsert);    

    // Query the Product table and display the results   
    newPoducts = connection.Query<Product>("SELECT top 3 * FROM Production.Product order by ProductID desc").ToList();

    foreach (var product in newPoducts)
    {
        Console.WriteLine($"{product.ProductID}, {product.Name}, {product.ProductNumber}, {product.Color}, {product.StandardCost}, {product.ListPrice}");
    }
}

1202, Gloves-7060, GL-H01-6285, Black, 5.0000, 9.9900
1201, Water Bottle-9321, WB-H01-5810, Blue, 1.0000, 3.4900
1200, Bike Helmet-5802, BK-H01-8188, Red, 15.5000, 34.9900


__Bulk update__:

In [5]:
var productsToUpdate = new List<Product>()
{
    new Product() { ProductID = newPoducts[0].ProductID, Name = "Adjustable Race-" + rand.Next(1000, 9999), 
                    ProductNumber = "AR-" + rand.Next(1000, 9999), Color = "Silver", StandardCost = 300.00m,
                    ListPrice = 400.00m, SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, 
                    SellStartDate = DateTime.Now },
    new Product() { ProductID = newPoducts[1].ProductID, Name = "Bearing Ball-" + rand.Next(1000, 9999), 
                    ProductNumber = "BA-" + rand.Next(1000, 9999), Color = "Black", StandardCost = 0.50m, 
                    ListPrice = 0.99m, SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, 
                    SellStartDate = DateTime.Now },
    new Product() { ProductID = newPoducts[2].ProductID, Name = "BB Ball Bearing-" + rand.Next(1000, 9999), 
                    ProductNumber = "BE-" + rand.Next(1000, 9999), Color = "Silver", StandardCost = 0.75m, 
                    ListPrice = 1.49m, SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, 
                    SellStartDate = DateTime.Now  }
};

using (var connection = new SqlConnection(connectionString))
{
    // Open the connection
    connection.Open();

    // Configure Dapper Plus to map the Product entity to the Product table and specify the identity column
    DapperPlusManager.Entity<Product>().Table("Production.Product").Identity(x => x.ProductID);
  
    // Perform bulk update operation
    connection.BulkUpdate(productsToUpdate);

    // Query the Product table and display the results
    var sql = $@"SELECT ProductID, Name, ProductNumber, Color, StandardCost, ListPrice, SafetyStockLevel, 
                        DaysToManufacture, SellStartDate
                        FROM Production.Product
                        WHERE ProductID = @ProductID";

    foreach(Product product in productsToUpdate)                    
    {
        List<Product> products = connection.Query<Product>(sql, product).ToList();
        foreach (var p in products)
        {
            Console.WriteLine($"{p.ProductID}, {p.Name}, {p.ProductNumber}, {p.Color}, {p.StandardCost}, {p.ListPrice}, {p.SafetyStockLevel}, {p.DaysToManufacture}, {p.SellStartDate}");
        }
    } 
}

1202, Adjustable Race-4394, AR-5732, Silver, 300.0000, 400.0000, 555, 0, 1/10/2024 8:45:46 PM
1201, Bearing Ball-2898, BA-4389, Black, 0.5000, 0.9900, 555, 0, 1/10/2024 8:45:46 PM
1200, BB Ball Bearing-5680, BE-1308, Silver, 0.7500, 1.4900, 555, 0, 1/10/2024 8:45:46 PM


__Bulk delete__:

In [6]:
var productsToDelete = new List<Product>()
{
    new Product() { ProductID = newPoducts[0].ProductID },
    new Product() { ProductID = newPoducts[1].ProductID },
    new Product() { ProductID = newPoducts[2].ProductID }
};

using (var connection = new SqlConnection(connectionString))
{
    // Open the connection
    connection.Open();

    // Configure Dapper Plus to map the Product entity to the Product table and specify the identity column
    DapperPlusManager.Entity<Product>().Table("Production.Product").Identity(x => x.ProductID);    

    // Perform bulk delete operation
    connection.BulkDelete(productsToDelete);

    // Query the Product table and display the results, which have been deleted already.
    var sql = $@"SELECT ProductID, Name, ProductNumber, Color, StandardCost, ListPrice, SafetyStockLevel, 
                        DaysToManufacture, SellStartDate
                        FROM Production.Product
                        WHERE ProductID = @ProductID";

    foreach(Product product in productsToUpdate)                    
    {
        List<Product> products = connection.Query<Product>(sql, product).ToList();
        foreach (var p in products)
        {
            Console.WriteLine($"{p.ProductID}, {p.Name}, {p.ProductNumber}, {p.Color}, {p.StandardCost}, {p.ListPrice}, {p.SafetyStockLevel}, {p.DaysToManufacture}, {p.SellStartDate}");
        }
    } 
}